# Battle of Neighborhoods

In this notebook I will be fetching location data using an API and using the data I will build a model to successfully perform clustering of neighborhoods with which are similar to eachother.

In [174]:
import pandas as pd
import numpy as np
print("Hello capstone project course!")

Hello capstone project course!


# 1) Webscraping and extracting neighborhood data
Import required libraries.

In [175]:
from bs4 import BeautifulSoup
import requests
import csv

In [176]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')

Parsing the required table from the website and converting it into a pandas dataframe.

In [177]:
stuff = soup.find('div', class_='mw-parser-output').table
data = pd.DataFrame()
rowi = 0
for row in stuff.findAll('tr'):
    coli = 0
    for col in row.findAll('td'):
        data.loc[rowi,coli] = col.get_text()
        coli += 1
    rowi += 1

data.columns = ['PostCode', 'Borough', 'Neighborhood']
data['Neighborhood'] = pd.Series(data['Neighborhood']).str.rstrip('\n')

Here I am removing data values where Borough has not been assgined.

In [178]:
data.drop(data[data['Borough'] == 'Not assigned'].index, inplace = True)
data.reset_index(drop = True, inplace = True)

Grouping Neighborhoods with same postal code and replaceing neighborhoods with borough values where it has not been assigned.

In [184]:
df = data.groupby(['PostCode'], as_index = False).agg({'Borough' : 'max', 'Neighborhood': ','.join}, inplace = True)
df['Neighborhood'] = np.where(df['Neighborhood'] == 'Not assigned', df['Borough'], df['Neighborhood'])
df.head()

,PostCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [185]:
df.shape

(103, 3)